In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import cv2 #opencv-python
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
%matplotlib inline
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.resnet50 import decode_predictions, ResNet50
import h5py
import datetime
import os
import shutil
import pathlib
import time
from IPython.display import display, Image, Markdown
import pickle
from numba import cuda

input = '/kaggle/input/dog-breed-identification/'
output = '/kaggle/working/'

def md(input):
    display(Markdown(input))

def step(input):
    return md(f"✅ *{input}*")


def save_model(model, model_path):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
#   modeldir = os.path.join("models",
#                           datetime.datetime.now().strftime("%d-%m-%y-%Hh%Mm"))
  print(f"Saving model to: {model_path}...")
  model.save(model_path)

def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path)
  return model
    
    
def plot_history(history):
    metrics = (('accuracy', 'val_accuracy'), ('loss', 'val_loss'))
    for metric in metrics:
        plt.plot(history[metric[0]])
        plt.plot(history[metric[1]])
        plt.title('model {}'.format(metric[0]))
        plt.ylabel(metric[0])
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

print("GPU", "available (YES!)" if tf.config.list_physical_devices("GPU") else "not available :(")
print("clear GPU memory...")
device = cuda.get_current_device()
device.reset() # clear GPU memory
step("Setup")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dir = "{}train/".format(input)
test_dir = "{}test/".format(input)
TRAINING_MODE = False # False = load model and history saved in file (much much faster) instead build it from zero
df = pd.read_csv("{}labels.csv".format(input))
df=df.sample(frac=1).reset_index(drop=True) #shuffle
df['path'] = df.id.apply(lambda x: '{}/{}.jpg'.format(train_dir, x)) # replace id by path to feed generator with flow_from_dataframe
df.drop('id', axis=1, inplace=True)
display(df.head())
step("Data loading")

On mélange le dataframe dès le début pour éviter de le faire par la suite ce qui pourrais compliquer notre interprétation des prédictions, notemment savoir quelle image est associée à chaque prédiction

## Exploration

In [ ]:
print("Nombre de photos des différentes races de chiens")
df["breed"].value_counts().plot.bar(figsize=(20, 10));

In [ ]:
{'nombre totale de chiens': df.shape[0], 'nombre de chiens de race la plus représentée': df["breed"].value_counts()[0], 'nombre de chiens de race la moins représentée': df["breed"].value_counts()[-1]}

In [ ]:
max_breed = df["breed"].value_counts().index[0]
min_breed = df["breed"].value_counts().index[-1]
random_max_breed = df.query("breed == @max_breed").sample()
random_min_breed = df.query("breed == @min_breed").sample()

In [ ]:
print("La classe la plus représentée : {} ({} images)".format(max_breed, df["breed"].value_counts()[0]))
Image(random_max_breed.path.item())

In [ ]:
print("La classe la moins représentée : {} ({} images)".format(min_breed, df["breed"].value_counts()[-1]))
Image(random_min_breed.path.item())

**Effectif de chaque race de chien**

In [ ]:
pd.DataFrame(df.breed.value_counts())

## Vérification des données

In [ ]:
from os import listdir
from os.path import isfile, join
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


sample_submission = pd.read_csv('{}sample_submission.csv'.format(input))

def get_all_files_in_dir(dir_path, full_path=True):
    if full_path:
        return [dir_path+f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    else:
        return [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    
train_image_paths = get_all_files_in_dir(train_dir)
test_image_paths = get_all_files_in_dir(test_dir)
md(f'Il y a **{len(train_image_paths):,}** images dans notre dossier train')
md(f'Il y a **{len(test_image_paths):,}** images dans notre dossier test')
if df.shape[0] != len(train_image_paths) or sample_submission.shape[0] != len(test_image_paths):
    print("/!\ Il y a une différence entre le nombre d'image est le nombre de lignes dans notre dataset!")
else:
    print("Il y autant d'images dans nos dossiers que de ligne dans nos dataset de train et de test.")
    

def get_img_infos(img):
    img_type = type(img)
    img_format = img.format
    img_mode = img.mode
    img_size = img.size
    return img_type, img_format, img_mode, img_size

sample_image = load_img(train_image_paths[0])
sample_type, sample_format, sample_mode, sample_size = get_img_infos(sample_image)



bad_img_count = 0
for image_paths in (train_image_paths, test_image_paths):
    for img_path in image_paths:
        img = load_img(img_path)
        img_type, img_format, img_mode, img_size = get_img_infos(img)
        if img_type != sample_type or img_format != sample_format or img_mode != sample_mode:
            print("L'image n'est pas conforme : {}".format(img_path))
            bad_img_count += 1
        
if bad_img_count == 0:
    md("**Toutes les images sont conformes**")
else:
    print("{} image(s) doivent être ajustées".format(bad_img_count))
    




In [ ]:
print("Voici une image pris au hasard:")
sample_image_array = img_to_array(sample_image)
plt.imshow(sample_image_array / 255.0);
print('Image type: {}'.format(sample_type))
print('Image format: {}'.format(sample_format))
print('Image mode: {}'.format(sample_mode))
print('Image size: {}'.format(sample_size))

In [ ]:
print("Pour pouvoir travailler avec des images on doit les convertir en chiffre c'est à dire en matrice. Chaque pixel devient un vecteur de taille 3 codant les couleurs RGB (red, green, blue)")
print('Transformation en matrice...')
print(f'Image type: {type(sample_image_array)}')
print(f'Image array shape: {sample_image_array.shape}')
sample_image_array

Après avoir explorer et vérifier les données passons à création de notre modèle de machine learning pour prédire les races de chien.

## Créations des tenseurs

In [ ]:
model_name = 'nasnet'
image_width = 331
image_size = (image_width, image_width)


generator = ImageDataGenerator(
    validation_split=0.02,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

# WITH AUGMENTATION
# generator = ImageDataGenerator(
#     validation_split=0.02,
#     horizontal_flip = True,
#     rotation_range = 20,
#     width_shift_range = 0.1,
#     height_shift_range = 0.1,
#     shear_range = 0.1,
#     zoom_range=0.1,
#     fill_mode = 'nearest',
#     preprocessing_function = preprocess_input
# )
        
train_generator = generator.flow_from_dataframe(
    dataframe=df,
    x_col="path",
    y_col="breed",
    target_size=image_size,
    batch_size=32,
    subset="training",
    shuffle=False,
)




valid_generator = generator.flow_from_dataframe(
    dataframe=df,
    x_col="path",
    y_col="breed",
    target_size=image_size,
    batch_size=32,
    subset="validation",
    shuffle=False,
)

step("Tensor generator")

On crée nos tenseurs à l'aide des générateurs de Tensorflow, comme nous disposons déjà d'un dataframe avec l'emplacement de nos fichiers et les labels associés on peut utiliser la fonctions **flow_from_dataframe**. On donne un nom à notre modèle qui servira à créer le fichier d'export du modèle. Toutes nos images doivent avoir **la même taille** pour avoir un traitement uniforme (un pixel = 3 données RGB). Si on a le choix on choisira 300x300 ou les recommandation du modèle pré-entrainé que l'on utilise. Il faut également **normaliser** les valeurs RGB pour cela on applique un rescale. Le batch size, c'est à dire la taille des paquet d'images qui seront soumises à chaque itération de notre entrainement est fixé à **32**, c'est la valeur qui marche la plupart du temps pour limiter le suraprentissage. **20%** de nos données seront dédiées à la validation. On peut ou pas choisir d'augmenter notre dataset à travers divers transformations (si vous décommentez il faut repasser un entrainement en passant TRAINING_MODE à True).

## Modèle

Si aucun fichier n'existe on entraine notre modèle puis on le sauvegarde ainsi que ses stats (history). Le modèle est optimisé pour maximiser l'accuracy, pour diminuer le loss on peut supprimmer le monitor="val_accuracy" et mettre patience=3 à l'early stopping, ainsi le modèle sera plus pertinent mais s'arretera plus tard et perdra un peu en précision.

In [ ]:
model_path = output + 'model_'+ model_name + ".h5"
history_path = output + 'history_'+ model_name + ".h5"

if not pathlib.Path(model_path).exists() or not pathlib.Path(history_path).exists() or TRAINING_MODE:
    md("**Il n'existe pas de fichiers, on doit entrainer notre modèle complétement une première fois**")
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2, 
        min_delta=0.001, 
        restore_best_weights=True
    )

    # Setup input shape to the model
    input_shape = [None, image_width, image_width, 3] # batch, height, width, colour channels

    # Setup output shape of the model
    output_shape = 120 # number of unique labels


    nas_model=NASNetLarge(
        include_top=False, 
        weights='imagenet', 
        input_shape=(image_width,image_width,3),
    )

    nas_model.trainable = False

    
    # Setup the model layers
    model = tf.keras.Sequential([
        nas_model,   
        layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(120, activation='softmax')
    ])


    # Compile the model
    opt = SGD(lr=1e-3, momentum=0.9)
    model.compile(
        optimizer = opt, 
        loss="categorical_crossentropy", 
        metrics=["accuracy"]
    )
    model.summary()

    STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
    history = model.fit(
        train_generator, 
        steps_per_epoch=STEP_SIZE_TRAIN, 
        validation_data=valid_generator, 
        validation_steps=STEP_SIZE_VALID, 
        epochs=25, 
        batch_size=32, 
        callbacks=[early_stopping], 
    )
    
    
    history = history.history
    pickle.dump(history, open( history_path, "wb" ) )
    save_model(model, model_path)
    
step("Making model")

## Evaluation

In [ ]:
model = load_model(model_path)
history = pickle.load(open(history_path, 'rb'))
def plot_history(history):
    metrics = (('accuracy', 'val_accuracy'), ('loss', 'val_loss'))
    for metric in metrics:
        plt.plot(history[metric[0]])
        plt.plot(history[metric[1]])
        plt.title('model {}'.format(metric[0]))
        plt.ylabel(metric[0])
        plt.xlabel('epoch')
        plt.legend(['train', 'valid'], loc='upper left')
        plt.show();
        if metric[0] == 'accuracy':
            md("best validation {} score: **{}**".format(metric[0], max(history[metric[1]])))
        else:
            md("best validation {} score: **{}**".format(metric[0], min(history[metric[1]])))
        
        
plot_history(history)
y_pred = model.predict(valid_generator, workers=16, verbose=1)
step("Evaluation")

## Prédictions sur les données de validation
A partir des prédictions de notre modèle on va construire un dataframe avec pour chaque prédiction l'image associée et les probabilitées attribuées à chaque race de chien.

In [ ]:
unique_breeds = list(pd.unique(df.breed))
unique_breeds.sort()
breed_pred = []
top10_pred = []
all_preds = []

for pred in y_pred:
    breed_pred.append(unique_breeds[np.argmax(pred)])
    top10_keys = pred.argsort()[-10:][::-1]
    top10_values = np.sort(pred)[-10:][::-1] 
    top10_pred.append(dict(zip([unique_breeds[key] for key in top10_keys], top10_values)))
    

df_pred = pd.DataFrame({'path':valid_generator.filenames, 'breed':[unique_breeds[label_index] for label_index in valid_generator.labels], 'pred': breed_pred, 'proba': top10_pred })
class_to_num = dict(zip(unique_breeds, range(120)))  # affenpinscher : 0
for name in unique_breeds:  
    df_pred[name] = y_pred[:,class_to_num[name]]

md("**Une ligne de notre dataframe de prédictions**")
display(df_pred.sample().T)
step("Predictions on validation data")

## Analyse des erreurs de prédictions
A partir de notre dataframe de prédictions on cherche à mieux visualiser les différentes érreurs de notre modèle


In [ ]:
def get_error(row):
    predictions = row.proba
    if row.pred != row.breed:
        best_score = list(predictions.values())[0]
        return best_score
    else:
        return 0

key_cols = ['path', 'breed', 'pred', 'proba']

df_pred = df_pred[key_cols].copy()
df_pred["error"] = df_pred.apply(get_error, axis=1)

error_counts = len(df_pred.query("pred != breed"))
errors_percent = round(error_counts/df_pred.shape[0]*100)
print("Nombre d'erreurs du modèle sur les données de validations : {}/{} ({}%)".format(error_counts,df_pred.shape[0], errors_percent))
print("Nous avons voulu savoir quelle prédictions avaient réalisé notre modèle lorsqu'il c'est trompé, pour cela on a filtrer les mauvaises prédictions et on a retenu le score de la plus forte prédiction éronnée pour produire deux classements différents. Le premier tableau montre les races de chiens les plus difficiles à prédire (moyenne de la marge d'érreur) tandis que le second montre les races qui détériorent le plus notre modèle (sommme au lieu de la moyenne)")
md("**les races les plus difficiles à prédire:**")
display(df_pred.groupby("breed").mean("error").sort_values(by=["error"],ascending=False)[0:20])
md("**les races dont les mauvaises prédictions impactent le plus notre modèle:**")
display(df_pred.groupby("breed").sum("error").sort_values(by=["error"],ascending=False)[0:20])


### Les images les plus difficiles à prédire
Voici les images où le modèle a le moins performé. La bonne réponse est en vert. La photo de gauche est l'image erronée et celle de droite est la race prédite. On constate que le problème ne vient pas de la qualité des images ou de leur driversité mais plutôt de la forte ressemblance que peuvent avoir différentes races de chien.

In [ ]:
md("**TRUE**{}**PRED**".format("&nbsp;"*75))

def plot_dog(row):
    breeds = list(row.proba.keys())
    scores = list(row.proba.values())
    pred_image_path = df.query("breed == @row.pred").sample().path.item()
    img = mpimg.imread(row.path)
    img = cv2.resize(img, (300, 300))
    pred_img = mpimg.imread(pred_image_path)
    pred_img = cv2.resize(pred_img, (300, 300))
    plt.figure(figsize=(20,5))
    Grid_plot = plt.GridSpec(1, 3, wspace=0.45)
    plt.subplot(Grid_plot[0, 0])
    plt.axis('off')
    imgplot = plt.imshow(img);
    plt.subplot(Grid_plot[0, 1])
    plt.axis('off')
    imgplot = plt.imshow(pred_img);    
    plt.subplot(Grid_plot[0, 2:])
    clrs = ['green' if x == row.breed else 'grey' for x in breeds]
    sns.barplot(y=breeds, x=scores, orient='h', palette=clrs)
    
    
df_pred.query("pred != breed").sort_values(by=["error"],ascending=False).apply(plot_dog, axis=1);

In [ ]:
step("Predictions details on validation data")

## Création du fichier de soumission
On utilise notre modèle pour prédire les races de chiens de notre échantillion de test et on enregistre ces prédictions dans un fichier que l'on pourra soumettre à Kaggle pour obtenir un score

In [ ]:
test_df = pd.read_csv('{}sample_submission.csv'.format(input))

new_id = [el +".jpg" for el in test_df["id"]]
test_df["id"] = new_id



test_datagen=ImageDataGenerator(
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col="id",
    y_col=None,
    target_size=image_size,
    batch_size=32,
    class_mode=None,
    shuffle=False,
)


y_pred = model.predict(test_generator)


df_sub = pd.read_csv('{}sample_submission.csv'.format(input))
display(df_sub.head())

df_sub.iloc[:,1:] = y_pred
display(df_sub.head())

final_df = df_sub.set_index('id')
filename = 'my_submission.csv'
final_df.to_csv(filename)
step("Fichier de soumission crée: {}".format(filename))